# Ping Tutorial
<i>Adapted for use with FABRIC from [Layer2Ping](https://groups.geni.net/geni/wiki/Tutorials/Layer2Ping/Procedure)  and [Lab Zero: A First Experiment Using GENI](https://groups.geni.net/geni/wiki/GENIExperimenter/Tutorials/jacks/GettingStarted_PartI/Procedure).
    
<img src="https://groups.geni.net/geni/attachment/wiki/GEC17Agenda/GettingStartedWithGENI_I/Graphics/2vmvlan_overview.png?format=raw" ><br>
In this tutorial you will learn:
* How to create a FABRIC slice and connect to nodes using SSH
* How to create a Layer 2 connection between nodes
    
<b> Prerequisites  
    
You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
  

## 1. Design the Experiment
In this section, be careful to do the instructions listed with **"Do this"**, as well as running the code blocks.

### 1.1 Reserve two virtual machines at one aggregate


<b>Do this:</b>
    Set bastion_username to your username in the block below.


In [ ]:
import os
# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+<INSERT_YOUR_BASTION_KEY>

# Set the keypair FABRIC will install in your slice.
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

#### Import the Fabric API

In [ ]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

#### Create slice

In [ ]:
slice_name = 'MyLayer2PingSlice'
site = 'MAX'

client_name = 'client'
server_name = 'server'

network_service_name='bridge1'

nic1_name = 'client-nic1'
nic2_name = 'server-nic1'

image = 'default_ubuntu_20'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 10

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(slice_name)

    # Add node
    client = slice.add_node(name=client_name, site=site)
    client.set_capacities(cores=cores, ram=ram, disk=disk)
    client.set_image(image)
    iface1 = client.add_component(model='NIC_Basic', name=nic1_name).get_interfaces()[0]
    
    # Add node
    server = slice.add_node(name=server_name, site=site)
    server.set_capacities(cores=cores, ram=ram, disk=disk)
    server.set_image(image)
    iface2 = server.add_component(model='NIC_Basic', name=nic2_name).get_interfaces()[0] 
    
    # Network
    net1 = slice.add_l2network(name=network_service_name, interfaces=[iface1, iface2])

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Failed: {e}")

#### Get the slice

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

### 1.2 Configure the Nodes

#### Get the topology

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   Interfaces        : {node.get_interfaces()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()                
except Exception as e:
    print(f"Fail: {e}")

#### Choose data interface IP addresses

In [ ]:
client_ip = '10.20.30.40'
server_ip = '10.20.30.41'

#### Configure client node

In [ ]:
try:
    client = slice.get_node(name=client_name)        
    iface1 = client.get_interface(network_name=network_service_name)  
    iface1.set_ip(ip=client_ip, cidr="24")
    
    stdout, stderr = client.execute(f'ip addr show {iface1.get_os_interface()}')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

#### Configure server node

In [ ]:
try:
    server = slice.get_node(name=server_name)        
    iface2 = server.get_interface(network_name=network_service_name)  
    iface2.set_ip(ip=server_ip, cidr="24")
    
    stdout, stderr = server.execute(f'ip addr show {iface2.get_os_interface()}')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

### 1.3 Get the ssh command

Add bastion ssh configuration to .ssh/config if it's not already added.

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()                
except Exception as e:
    print(f"Fail: {e}")

## 2. Execute the experiment

From here on you will be working directly from the terminal.
### 2.1 Open terminal windows
Open two terminal windows by clicking the blue "+" to open the launcher, selecting "Terminal," and repeating.
### 2.2 SSH to nodes
Use the ssh commands from the last code block to login to the server in one terminal and the client in the other terminal.
### 2.3 Check IP address
In each terminal, type `ip addr show`. You should see at least two interfaces:
* The control interface. This is the interface you use to access the node, e.g. by ssh. The control interface is mainly used for control traffic (likely something like 10.20.4.18 on ens3).
 * The data interface. This is the interface that is used for sending experimental traffic and connects to the other nodes of your experiment through FABRIC. The links between these interfaces are the ones that allow you to run non-IP experiments. The data interface is the one that has an IP address and mask that match what you configured before you reserved your resources (10.20.30.40 for the client and 10.20.30.41 for the server, both on ens7).   
 
Write down the ip address and name for the control and data interfaces on each node.  

### 2.4 Verify you can connect between nodes
 We will use ping to do this.  
    In the server terminal, type `ping -c 3 10.20.30.40`  
    In the client terminal, type `ping -c 3 10.20.30.41`  
If the pings succeed you know you have IP connectivity between the two nodes.  

### 2.5 Install necessary software
Run the below commands on both nodes

        sudo apt-get update
        sudo apt-get install iperf  
        sudo apt-get install gcc
        sudo apt-get install make

### 2.6 Measure the bandwidth

* Start an iperf server on the server node:
        iperf -s


* Run an iperf client via the data interface:
        iperf -c <server data IP addr>  
    
    
    * What is the bandwidth of this link? Why?  

* Run an iperf client via the control plane:
        iperf -c <server control IP addr>  

    * What is the bandwidth of this link? Why?  

* Type Ctrl+C on the server node to stop the iperf server. 

### 2.7 Download and install PingPlus on the server and client nodes
We will download and install a program that does Layer 2 pings on the server and client nodes. Do this on both nodes:  

``` 
wget https://github.com/GENI-NSF/geni-tutorials/raw/master/PingPlus/pingPlus-0.2.tar.gz  
tar xvfz pingPlus-0.2.tar.gz
```
``` 
wget https://github.com/smoser82/geni-tutorials/raw/main/pingPlus-0.2.1.tar.gz
tar xvfz pingPlus-0.2.1.tar.gz
```
You should see a directory called pingPlus-0.2. This directory has the source code for pingPlus.  
### 2.8 Compile pingPlus
Compile pingPlus on both nodes:

```
cd pingPlus-0.2  
make  
```
    
### 2.9 Turn off IP on the data interface
We now turn off IP on the data interface. Be very careful to not turn off IP on any of the other interfaces as this will make your node unreachable.
```
sudo apt install net-tools
sudo /sbin/ifconfig <data_interface_name> 0.0.0.0 
```
    
### 2.10 Run pingPlus 

In the server terminal, run:

`./pingPlusListener 10002`

The 10002 is the value we'll use for the type field in the header of the Ethernet frames we send and receive. The above command tells the server to look for Ethernet packets of this type. (In this exercise you can use almost any value for this type field.)

In the client terinal, run:

`./pingPlus <server_data_interface_hardware_addr> <client_data_interface_name> 10002`

This tells the client to send an Ethernet frame with destination address set to the server's data interface MAC address and with the type field set to 10002. This frame will be sent out of the specified interface on the client.

The client prints out the contents of the data field of the frame it sent to the server and the contents of the data field in the frame it got back from the server.

In the server terminal, kill the pingPlusListener by typing Ctrl+C.
    
### 2.11 Examine the pingPlus program

Open up an editor (vi or emacs) to view pingPlus.c. All the interesting stuff happens starting line 110 of this file:

* Open a Layer 2 socket (lines 110 - 115)
* Bind the socket to the specified interface (lines 117 - 125)
* In a buffer compose the contents of the data portion of the Ethernet frame to be sent to the server. In our case the data portion has the format "RQ:<randomNumber>+<randomNumber>" (line 134)
* Send the packet. The send is successful if the return from sendPacket matches the number of bytes sent. The sendPacket function is implemented in the file packetFunctions.c in this directory. This function sets the appropriate header fields in the ethernet frame before sending it.
* Wait for the server to respond (line 145). The receivePacket function is implemented in file packetFunctions.c. 

### 2.12 Modify pingPlus to measure round-trip delays

You should by now have a feel for how Layer 2 sockets work. You can now modify ping2Plus to measure round-trip delays similar to the Layer 3 ping. You will want to:

* get the current time before you send the packet using
* get the current time when you get a response from the server. 

### 2.13 (Optional) Get a solution to the exercise

To see a solution to this exercise, create a new directory and dowload the solution to that directory:

```
cd 
wget https://github.com/GENI-NSF/geni-tutorials/raw/master/PingPlus/pingPlus_v3.tar.gz         
tar xvfz pingPlus_v3.tar.gz
cd pingPlus_v3
```

The file pingPlus.c in this directory will have the solution. The statements that do the timing measurement are around lines 151 and 175.


## 3. Finish!

Delete your slice

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")